In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
import os
import zipfile
from os import path, getcwd, chdir

In [2]:
# filepath
path = "./data/happy-or-sad.zip"

zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("./data/h-or-s")
zip_ref.close()

This function demonstrates how to read images from a local directory and build a binary classification model.

In [3]:
def train_happy_sad_model():

    DESIRED_ACCURACY = 0.999

    # modelCallback class
    class modelCallback(tf.keras.callbacks.Callback):
         # Your Code
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('acc')>=DESIRED_ACCURACY):
                print('\nDesired Accuracy reached. Stopping model training!')
                self.model.stop_training = True
    
    # Instantiates the modelCallback class
    callbacks = modelCallback()
    
    # Model definition
    model = tf.keras.models.Sequential([
        # Your Code Here
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu',
                              input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        # Flatten layer
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    # Model compilation
    model.compile(loss='binary_crossentropy',
                  # RMSProp supports adjustable lr
                  optimizer=RMSprop(lr=0.001),
                  metrics=['acc'])
    
    # Make data generator function
    def make_data_generator():
        train_dataset_folder = './data/h-or-s'
        train_datagen = ImageDataGenerator(rescale=1/255)
        train_generator = train_datagen.flow_from_directory(train_dataset_folder, 
                                                            target_size=(150, 150), 
                                                            class_mode='binary', 
                                                            batch_size=128)
        return train_generator

    # Train dataset
    train_dataset = tf.data.Dataset.from_generator(make_data_generator,
                                                   output_types=(tf.float32, tf.float32),
                                                   output_shapes=(tf.TensorShape((None, None, None, None)), 
                                                                  tf.TensorShape((80, ))))
    
    # Model fitting
    history = model.fit(
        train_dataset,
        steps_per_epoch=8,
        epochs=15,
#         validation_data=None,
#         validation_steps=8,
        # Defines how much data should be displayed during training
        verbose=2,
        callbacks=[callbacks]
    )

    return history.history['acc'][-1]

In [4]:
train_happy_sad_model()

Train for 8 steps
Epoch 1/15
Found 80 images belonging to 2 classes.
8/8 - 7s - loss: 2.9149 - acc: 0.5078
Epoch 2/15
8/8 - 6s - loss: 0.3737 - acc: 0.8422
Epoch 3/15
8/8 - 6s - loss: 0.1804 - acc: 0.9625
Epoch 4/15
8/8 - 6s - loss: 0.1301 - acc: 0.9641
Epoch 5/15
8/8 - 6s - loss: 0.0554 - acc: 0.9797
Epoch 6/15
8/8 - 6s - loss: 0.0310 - acc: 0.9937
Epoch 7/15

Desired Accuracy reached. Stopping model training!
8/8 - 6s - loss: 0.0117 - acc: 1.0000


1.0